In [1]:
import numpy as np
import pandas as pd
import json
import re
import string

from nltk.corpus import stopwords

In [ ]:
def to_dict(string):
    if string != "[]":
        string = json.loads(string.replace("'", "\""))
        return ",".join([s["screen_name"] for s in string])
    return ""

def to_list(list_):
    if list_ != "[]":
        list_ = list_[1:-1]
        list_ = list_.split(",")
        return ",".join([s.strip().strip("'") for s in list_])
    return ""

def normalize(s):
    replacements = (("á", "a"), ("é", "e"), ("í", "i"), ("ó", "o"), ("ú", "u"))
    for a, b in replacements:
        s = s.lower()
        s = s.replace(a, b)
    return s

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r"", text)

def cleanTxt(text):
    text = re.sub(r"@[a-zA-Z0-9]+", "", text) #Removes @mentions
    text = re.sub(r"#", "", text) #Removing the "#" symbol
    text = re.sub(r"RT[\s]+", "", text) #Removing RT
    text = re.sub(r"https?:\/\/\S+", "", text) #Remove the hyperlink
    return text

def replace_punct(s):
    for i in string.punctuation:
        if i in s:
            s = s.replace(i, "").strip()
    return s

def replace_num(s):
    for i in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
        s = s.replace(i, "")
    return s

def tokenizador(text):
    important_words = []
    for word in text.split(" "):
        if word not in stopwords.words("spanish"):
            if word != "":
                important_words.append(word)
    return " ".join(important_words).strip()

def foo(text):
    forbidden = ("?", "¿", "¡", "!", ",", ".", ";", ":", "-", "'", "+", "$", "/", "*",'«','»', "~", "(", ")")
    aux = ""
    for v in text:
        if not v in forbidden:
            aux += v
    return aux

def quita_palabras_pequeñas(text):
    return " ".join([word for word in text.split(" ") if len(word) > 2])  

In [2]:
df_c = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/trends/23_24f.csv")

C:\Users\Daniel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_d = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/trends/24_25f.csv")

In [4]:
df_e = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/trends/24f_pilar.csv")

C:\Users\Daniel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,22,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_f = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/trends/dia_24_tendencias_daniel.csv", sep = ";")

C:\Users\Daniel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_g = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/trends/24f_ismael.csv")

In [7]:
df = pd.concat([df_c, df_d, df_e, df_f, df_g])

In [ ]:
df.drop_duplicates(inplace = True, ignore_index = True)

In [ ]:
df.to_csv("tweets_24_tendencias_raw_daniel.csv", sep = ";", index = True)

In [ ]:
columns_to_drop = ["conversation_id", "cashtags", "timezone", "user_id", "name", "near", "geo", "source",
                   "user_rt_id", "user_rt", "retweet_id", "retweet_date", "translate", "trans_src",
                   "trans_dest", "place", "quote_url", "thumbnail", "created_at", "id", "link"]

df.drop(columns_to_drop, axis = 1, inplace = True)

df = df[df.language == "es"]

df.drop("language", axis = 1, inplace = True)

df = df.reset_index().drop("index", axis = 1)

In [ ]:
df["day"] = df.date.apply(lambda x : x[-2:])
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [ ]:
df = df[df.day == "24"]
df = df.reset_index().drop("index", axis = 1)

In [ ]:
reply_to_rows = []
for num, row in enumerate(df.reply_to):
    try:
        to_dict(row)
    except:
        reply_to_rows.append(num)

print("indices a dropear:", len(reply_to_rows))
        
df.drop(reply_to_rows, inplace = True)

df.reply_to = df.reply_to.apply(to_dict)

df = df.reset_index().drop("index", axis = 1)

In [ ]:
mention_rows = []
for num, row in enumerate(df.mentions):
    try:
        to_dict(row)
    except:
        mention_rows.append(num)
        
print("indices a dropear:", len(mention_rows))
        
df.drop(mention_rows, inplace = True)

df.mentions = df.mentions.apply(to_dict)

df = df.reset_index().drop("index", axis = 1)

In [ ]:
hashtags_rows = []
for num, row in enumerate(df.hashtags):
    try:
        to_list(row)
    except:
        hashtags_rows.append(num)
        
print("indices a dropear:", len(hashtags_rows))
        
df.drop(hashtags_rows, inplace = True)

df.hashtags = df.hashtags.apply(to_list)

df = df.reset_index().drop("index", axis = 1)

In [ ]:
df.photos = df.photos.apply(lambda x : 1 if x != "[]" else 0)
df.retweet = df.retweet.apply(lambda x : 1 if x == "True" else 0)
df.urls = df.urls.apply(lambda x : 1 if x != "[]" else 0)

In [ ]:
df.drop_duplicates(inplace = True, ignore_index = True)

In [ ]:
df["month"] = df.date.apply(lambda x : x[5 : 7])
df["day"] = df.date.apply(lambda x : x[-2:])

df["hour"] = df.time.apply(lambda x : x[:2])
df["minute"] = df.time.apply(lambda x : x[3:5])
df["second"] = df.time.apply(lambda x : x[6:])

In [ ]:
df["mentions_count"] = [len(mention.split(",")) if type(mention) == str else 0 for mention in df.mentions]

df["reply_to_count"] = [len(reply.split(",")) if type(reply) == str else 0 for reply in df.reply_to]

df["hashtags_count"] =  [len(hashtag.split(",")) if type(hashtag) == str else 0 for hashtag in df.hashtags]

df["interaccion"] = [rt + re + lk for rt, re, lk in zip(df.retweets_count, df.replies_count, df.likes_count)]

In [ ]:
%%time
df.tweet = df.tweet.apply(normalize)
df.tweet = df.tweet.apply(deEmojify)
df.tweet = df.tweet.apply(cleanTxt)
df.tweet = df.tweet.apply(replace_punct)
df.tweet = df.tweet.apply(replace_num)

df.tweet = df.tweet.apply(tokenizador)
df.tweet = df.tweet.apply(foo)
df.tweet = df.tweet.apply(quita_palabras_pequeñas)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
for trend in df.trend.unique():
    print(trend)

In [ ]:
df.to_csv("tweets_24_tendencias_preprocesado.csv", sep =";", index = False)

In [ ]:
df_raw = pd.concat([df_c, df_d, df_e, df_f, df_g])

In [ ]:
df_raw.drop_duplicates(inplace = True, ignore_index = True)

In [ ]:
df_raw.to_csv("tweets_24_tendencias_raw.csv", sep = ";", index = False)

In [ ]:
df_summary = pd.DataFrame(df.trend.value_counts()).reset_index()
df_summary.columns = ["trend", "total_tweet"]

In [ ]:
%%time

df_summary["total_hashtags"]    = [df[df.trend == trend].hashtags_count.sum() for trend in df_summary.trend]

df_summary["total_mentions"]    = [df[df.trend == trend].mentions_count.sum() for trend in df_summary.trend]

df_summary["total_reply_to"]    = [df[df.trend == trend].reply_to_count.sum() for trend in df_summary.trend]

df_summary["total_url"]         = [df[df.trend == trend].urls.sum()           for trend in df_summary.trend]

df_summary["total_photo"]       = [df[df.trend == trend].photos.sum()         for trend in df_summary.trend]

df_summary["total_retweets"]    = [df[df.trend == trend].retweets_count.sum() for trend in df_summary.trend]

df_summary["total_likes"]       = [df[df.trend == trend].likes_count.sum()    for trend in df_summary.trend]

df_summary["total_replies"]     = [df[df.trend == trend].replies_count.sum()  for trend in df_summary.trend]

df_summary["total_interaction"] = [df[df.trend == trend].interaccion.sum()    for trend in df_summary.trend]

In [ ]:
lifetime = []
for trend in df_summary.trend:
    start_end = list(df[df.trend == trend].sort_values("time").time.iloc[[0, -1]])
    start_end = int(start_end[1][0:2]) - int(start_end[0][0:2])
    lifetime.append(start_end)
    
df_summary["lifetime"] = lifetime

In [ ]:
time_intervals = ["0{}:00:00".format(i) if i < 10 else "{}:00:00".format(i) for i in range(24)] + ["23:59:59"]

tweet_count_labels = ["tweet_count_{}{}".format(i, i + 1) for i in range(24)]

for num, label in enumerate(tweet_count_labels):
    df_summary[label] = [df[(df.time.between(time_intervals[num], time_intervals[num + 1])) & (df.trend == trend)].shape[0] for trend in df_summary.trend]

In [ ]:
tweet_vel_labels = ["tweet_vel_{}{}".format(i, i + 1) for i in range(24)]

velocidades = []
for i in range(df_summary.shape[0]):
    counts = [0] + list(df_summary.loc[i, tweet_count_labels])
    vel = [counts[i] - counts[i - 1] for i in range(1, len(counts))]
    velocidades.append(vel)
    
velocidades = np.array(velocidades)

df_summary = pd.concat([df_summary, pd.DataFrame(velocidades, columns = tweet_vel_labels)], axis = 1)

In [ ]:
tweet_acl_labels = ["tweet_acl_{}{}".format(i, i + 1) for i in range(24)]

aceleraciones = []
for i in range(df_summary.shape[0]):
    counts = [0] + list(df_summary.loc[i, tweet_vel_labels])
    acl = [counts[i] - counts[i - 1] for i in range(1, len(counts))]
    aceleraciones.append(acl)
    
aceleraciones = np.array(aceleraciones)

df_summary = pd.concat([df_summary, pd.DataFrame(aceleraciones, columns = tweet_acl_labels)], axis = 1)

In [ ]:
user_count_labels = ["user_count_{}{}".format(i, i + 1) for i in range(24)]

for num, label in enumerate(user_count_labels):
    df_summary[label] = [df[(df.time.between(time_intervals[num], time_intervals[num + 1])) & (df.trend == trend)].username.unique().shape[0] for trend in df_summary.trend]

In [ ]:
df_summary.to_csv("tweets_24_tendencia_variables.csv", sep = ";", index = False)